In [ ]:
!pip install mne

In [ ]:
import mne
import numpy as np
import pandas as pd

psg_file_path = '/content/ST7011J0-PSG.edf'
hypnogram_file_path = '/content/ST7011JP-Hypnogram.edf'

psg_data = mne.io.read_raw_edf(psg_file_path, preload=True, verbose=False)

hypnogram_data = mne.read_annotations(hypnogram_file_path)

psg_data.set_annotations(hypnogram_data, emit_warning=False)

psg_data_info = psg_data.info

annotations_desc = hypnogram_data.description
annotations_onset = hypnogram_data.onset

(psg_data_info, annotations_desc, annotations_onset)



(<Info | 8 non-empty values
  bads: []
  ch_names: EEG Fpz-Cz, EEG Pz-Oz, EOG horizontal, EMG submental, Marker
  chs: 5 EEG
  custom_ref_applied: False
  highpass: 0.0 Hz
  lowpass: 800.0 Hz
  meas_date: 1994-07-12 23:00:00 UTC
  nchan: 5
  projs: []
  sfreq: 100.0 Hz
  subject_info: 4 items (dict)
 >,
 array(['Sleep stage W', 'Sleep stage 1', 'Sleep stage 2', 'Sleep stage 1',
        'Sleep stage 2', 'Sleep stage 3', 'Sleep stage 1', 'Sleep stage 2',
        'Sleep stage 3', 'Sleep stage 2', 'Sleep stage 1', 'Sleep stage 2',
        'Sleep stage W', 'Sleep stage 1', 'Sleep stage 2', 'Sleep stage W',
        'Sleep stage 2', 'Sleep stage 3', 'Sleep stage 2', 'Sleep stage 1',
        'Sleep stage W', 'Sleep stage 1', 'Sleep stage 2', 'Sleep stage 1',
        'Sleep stage 2', 'Sleep stage 1', 'Sleep stage W', 'Sleep stage 1',
        'Sleep stage 2', 'Sleep stage 3', 'Sleep stage 2', 'Sleep stage 3',
        'Sleep stage 2', 'Sleep stage R', 'Sleep stage 2', 'Sleep stage 1',
        'Sl

In [ ]:
import numpy as np
eeg_data = psg_data.get_data(picks=['EEG Fpz-Cz'])[0]
features = []
labels = []
for desc, onset in zip(annotations_desc, annotations_onset):
    start_sample = int(onset * psg_data.info['sfreq'])
    try:
        next_onset_index = list(annotations_onset).index(onset) + 1
        end_sample = int(annotations_onset[next_onset_index] * psg_data.info['sfreq'])
    except IndexError:
        end_sample = len(eeg_data)

    segment = eeg_data[start_sample:end_sample]
    mean = np.mean(segment)
    std = np.std(segment)

    features.append([mean, std])
    labels.append(desc)

features[:5], labels[:5]


([[1.269960888162266e-07, 4.084959622090936e-05],
  [-7.720604691855796e-07, 1.6782008799526415e-05],
  [4.6985803641125744e-07, 3.355460495604978e-05],
  [1.7860587194042185e-06, 1.997308016664745e-05],
  [-3.5052798632729462e-06, 1.9026291544765577e-05]],
 ['Sleep stage W',
  'Sleep stage 1',
  'Sleep stage 2',
  'Sleep stage 1',
  'Sleep stage 2'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
X_train, X_test, y_train, y_test = train_test_split(
    features, encoded_labels, test_size=0.3, random_state=42
)

X_train_shape = np.array(X_train).shape
X_test_shape = np.array(X_test).shape
y_train_shape = np.array(y_train).shape
y_test_shape = np.array(y_test).shape

X_train_shape, X_test_shape, y_train_shape, y_test_shape


((161, 2), (70, 2), (161,), (70,))

In [ ]:

clean_features = np.array(features)
clean_labels = np.array(labels)

nan_indices = np.where(np.isnan(clean_features).any(axis=1))

clean_features = np.delete(clean_features, nan_indices, axis=0)
clean_labels = np.delete(clean_labels, nan_indices)

encoded_labels_clean = label_encoder.fit_transform(clean_labels)

X_train_clean, X_test_clean, y_train_clean, y_test_clean = train_test_split(
    clean_features, encoded_labels_clean, test_size=0.3, random_state=42
)
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_clean, y_train_clean)


RandomForestClassifier(random_state=42)

In [ ]:
y_pred_clean = rf_classifier.predict(X_test_clean)

In [ ]:
unique_labels = np.unique(np.concatenate((y_test_clean, y_pred_clean)))
classification_rep_corrected = classification_report(y_test_clean, y_pred_clean, labels=unique_labels, target_names=label_encoder.inverse_transform(unique_labels))

classification_rep_corrected_dict = classification_report(y_test_clean, y_pred_clean, labels=unique_labels, target_names=label_encoder.inverse_transform(unique_labels), output_dict=True)
classification_rep_corrected_df = pd.DataFrame(classification_rep_corrected_dict).transpose()

classification_rep_corrected_df


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

,precision,recall,f1-score,support
Sleep stage 1,0.500000,0.384615,0.434783,13.000000
Sleep stage 2,0.425000,0.708333,0.531250,24.000000
Sleep stage 3,0.625000,0.250000,0.357143,20.000000
Sleep stage 4,0.166667,0.500000,0.250000,2.000000
Sleep stage R,0.000000,0.000000,0.000000,4.000000
Sleep stage W,0.166667,0.142857,0.153846,7.000000
accuracy,0.414286,0.414286,0.414286,0.414286
macro avg,0.313889,0.330968,0.287837,70.000000
weighted avg,0.438571,0.414286,0.387456,70.000000


In [ ]:
from scipy.signal import welch

def compute_psd_features(signal_segment, sfreq):
    freq_bands = {
        'Delta': (0.5, 4),
        'Theta': (4, 8),
        'Alpha': (8, 12),
        'Beta': (12, 30),
    }
    psd_features = []

    freqs, psd = welch(signal_segment, fs=sfreq, nperseg=sfreq*2)

    for band in freq_bands.values():
        idx_band = np.logical_and(freqs >= band[0], freqs <= band[1])
        band_power = np.sum(psd[idx_band])
        psd_features.append(band_power)

    return psd_features

psd_features_list = []
for desc, onset in zip(annotations_desc, annotations_onset):
    start_sample = int(onset * psg_data.info['sfreq'])
    try:
        next_onset_index = list(annotations_onset).index(onset) + 1
        end_sample = int(annotations_onset[next_onset_index] * psg_data.info['sfreq'])
    except IndexError:
        end_sample = len(eeg_data)

    segment = eeg_data[start_sample:end_sample]
    if segment.size > 0:
        psd_features = compute_psd_features(segment, sfreq=psg_data.info['sfreq'])
        psd_features_list.append(psd_features)
    else:
        psd_features_list.append([np.nan, np.nan, np.nan, np.nan])

clean_psd_features = np.array(psd_features_list)
clean_psd_features = np.delete(clean_psd_features, nan_indices, axis=0)

combined_features = np.hstack((clean_features, clean_psd_features))

X_train_psd, X_test_psd, y_train_psd, y_test_psd = train_test_split(
    combined_features, encoded_labels_clean, test_size=0.3, random_state=42
)

combined_features.shape


(231, 6)

In [ ]:
rf_classifier.fit(X_train_psd, y_train_psd)

y_pred_psd = rf_classifier.predict(X_test_psd)
accuracy_psd = accuracy_score(y_test_psd, y_pred_psd)
print(f"Accuracy: {accuracy_psd}")

classification_rep_psd_dict = classification_report(y_test_psd, y_pred_psd, target_names=label_encoder.inverse_transform(np.unique(y_test_psd)), output_dict=True)
classification_rep_psd_df = pd.DataFrame(classification_rep_psd_dict).transpose()

classification_rep_psd_df



Accuracy: 0.4142857142857143


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,precision,recall,f1-score,support
Sleep stage 1,0.416667,0.384615,0.400000,13.000000
Sleep stage 2,0.425000,0.708333,0.531250,24.000000
Sleep stage 3,0.714286,0.250000,0.370370,20.000000
Sleep stage 4,0.166667,0.500000,0.250000,2.000000
Sleep stage R,0.000000,0.000000,0.000000,4.000000
Sleep stage W,0.200000,0.142857,0.166667,7.000000
accuracy,0.414286,0.414286,0.414286,0.414286
macro avg,0.320437,0.330968,0.286381,70.000000
weighted avg,0.451939,0.414286,0.386058,70.000000


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Conv1D(filters=256, kernel_size=3, activation='relu', input_shape=(X_train_psd.shape[1], 1), padding='same'),
    BatchNormalization(),
    Dropout(0.3),
    Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'),
    BatchNormalization(),
    Dropout(0.3),
    Flatten(),
    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=7, min_lr=0.0001)

history = model.fit(
    X_train_psd_reshaped, y_train_cat,
    epochs=200,
    batch_size=16,
    verbose=1,
    validation_split=0.2,
    callbacks=[early_stopping, reduce_lr]
)

test_loss, test_acc = model.evaluate(X_test_psd_reshaped, y_test_cat, verbose=1)
print(f"Test Accuracy: {test_acc}")


Epoch 1/200
6/6 [==============================] - 3s 161ms/step - loss: 3.8255 - accuracy: 0.1765 - val_loss: 3.5776 - val_accuracy: 0.2727 - lr: 0.0010
Epoch 2/200
6/6 [==============================] - 0s 27ms/step - loss: 3.4157 - accuracy: 0.2941 - val_loss: 3.2806 - val_accuracy: 0.2727 - lr: 0.0010
Epoch 3/200
6/6 [==============================] - 0s 29ms/step - loss: 3.1232 - accuracy: 0.3059 - val_loss: 3.0498 - val_accuracy: 0.2727 - lr: 0.0010
Epoch 4/200
6/6 [==============================] - 0s 27ms/step - loss: 2.9079 - accuracy: 0.2471 - val_loss: 2.8680 - val_accuracy: 0.2727 - lr: 0.0010
Epoch 5/200
6/6 [==============================] - 0s 18ms/step - loss: 2.7095 - accuracy: 0.3059 - val_loss: 2.7242 - val_accuracy: 0.2727 - lr: 0.0010
Epoch 6/200
6/6 [==============================] - 0s 19ms/step - loss: 2.5883 - accuracy: 0.2588 - val_loss: 2.6056 - val_accuracy: 0.2727 - lr: 0.0010
Epoch 7/200
6/6 [==============================] - 0s 15ms/step - loss: 2.4843 - 